### 02-Movie-Clean.ipynb
* **Script:** 02-Movie-Clean.ipynb
* **Purpose:** Clean scraped movie data and engineer features
* **Input:** data/movie_data_raw_YYYY.pickle
* **Output:** data/movie_data_clean.pickle

In [1]:
import numpy as np
import pandas as pd

#### Read in Pickled List of Dicts with movie data from each year, append into a single pandas dataframe

In [2]:
def create_dataframe(start_year, stop_year):
    data=[]
    
    for yr in range(start_year,(stop_year+1)):
        pickle=pd.read_pickle("data/movie_data_raw_"+str(yr)+".pickle")
        data+=pickle
        
    df=pd.DataFrame(data)
    df=df[(df["release_year"]>=start_year) & (df["release_year"]<=stop_year)]
    
    return df
        
df=create_dataframe(2005, 2015)

In [3]:
print "Number of Records: ", df.shape[0]
print ""
print "Data Columns: ", df.columns
print ""
print df.info()
print ""
df.head(5)

Number of Records:  6797

Data Columns:  Index([u'budget', u'genre', u'imdb_id', u'imdb_rating', u'metascore',
       u'oscar_noms', u'oscar_wins', u'plot', u'rating', u'release_date',
       u'release_year', u'runtime', u'title', u'tomato_meter',
       u'tomato_user_meter'],
      dtype='object')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6797 entries, 0 to 6873
Data columns (total 15 columns):
budget               1491 non-null float64
genre                6797 non-null object
imdb_id              3999 non-null object
imdb_rating          3999 non-null object
metascore            3999 non-null object
oscar_noms           6797 non-null int64
oscar_wins           6797 non-null int64
plot                 3999 non-null object
rating               6797 non-null object
release_date         6797 non-null datetime64[ns]
release_year         6797 non-null int64
runtime              6574 non-null float64
title                6797 non-null object
tomato_meter         3999 non-null obje

,budget,genre,imdb_id,imdb_rating,metascore,oscar_noms,oscar_wins,plot,rating,release_date,release_year,runtime,title,tomato_meter,tomato_user_meter
0,26000000,Comedy,tt0405422,7.2,73,0,0,"Goaded by his buddies, a nerdy guy who's never...",R,2005-08-19,2005,116,The 40-Year-Old Virgin,85,84
1,62000000,Sci-Fi Action,tt0402022,5.5,36,0,0,Aeon Flux is a mysterious assassin working for...,PG-13,2005-12-02,2005,95,Aeon Flux,10,39
2,19000000,Horror,tt0384806,6.0,33,0,0,Newlyweds are terrorized by demonic forces aft...,R,2005-04-15,2005,89,The Amityville Horror,23,52
3,32000000,Family Comedy,tt0368578,4.5,27,0,0,Nick persons is a rich but selfish player who ...,PG,2005-01-21,2005,94,Are We There Yet?,11,45
4,35000000,Sports Comedy,tt0408524,5.8,65,0,0,A grizzled little league coach tries to turn h...,PG-13,2005-07-22,2005,111,Bad News Bears,N/A,0


#### Pool Similar Genres into More General Categories (by hand)

In [4]:
#df["genre"].value_counts().to_csv("data/genre_counts.csv") #Output list of unique genres to csv for hand coding
genre_coded=pd.read_csv("data/genre_counts_coded.csv") #Read hand coded list back in
df=pd.merge(df, genre_coded, on="genre") #join back onto main DF

print df["genre_group"].value_counts(dropna=False)

Drama                       1394
Foreign                     1241
Comedy                      1174
Documentary                 1121
Horror/ Thriller/ Sci-Fi     799
Action/ Adventure            437
Other                        328
Family/ Animation            284
NaN                           19
Name: genre_group, dtype: int64


#### Group Ratings into G/PG, PG-13, R/NC17, and Unrated

In [5]:
df.ix[df["rating"]=="Unknown", "rating"]=np.nan
df.ix[df["rating"].isin(["G","PG"]), "rating"]="G/ PG"
df.ix[df["rating"].isin(["R","NC-17"]), "rating"]="R/ NC17"

print df["rating"].value_counts(dropna=False)

Unrated    2649
R/ NC17    2104
PG-13      1360
G/ PG       682
NaN           2
Name: rating, dtype: int64


#### Clean numeric variables (convert to numeric, set non-numeric entries to NaN)

In [6]:
def clean_ratings(var):    
    df[var]=pd.to_numeric(df[var], errors="coerce")
    
clean_ratings("imdb_rating")
clean_ratings("metascore")
clean_ratings("tomato_meter")
clean_ratings("tomato_user_meter")

#### Create a feature showing which quarter of the year a film was released in

In [7]:
df["release_qtr"]=0
df.ix[df["release_date"].dt.month.isin([1,2,3]), "release_qtr"]=1
df.ix[df["release_date"].dt.month.isin([4,5,6]), "release_qtr"]=2
df.ix[df["release_date"].dt.month.isin([7,8,9]), "release_qtr"]=3
df.ix[df["release_date"].dt.month.isin([10,11,12]), "release_qtr"]=4

print df["release_qtr"].value_counts(dropna=False)

3    1822
2    1739
4    1722
1    1514
Name: release_qtr, dtype: int64


#### Clean Production Budget

In [8]:
df["budget"]=df.budget/1000 #normalize budget in thousands 

df.ix[df.budget<1800, "budget"]=np.nan #trim lower outliers (roughly bottom 5th percentile)
df.ix[df.budget>142000, "budget"]=np.nan #trim upper outliers (roughly top 5th percentile)

print df["budget"].describe()

count      1338.000000
mean      38100.590994
std       33426.610041
min        1884.000000
25%       13791.850000
50%       27210.000000
75%       52945.000000
max      141484.000000
Name: budget, dtype: float64


#### Pickle cleaned data for modelling

In [9]:
df.to_pickle("data/movie_data_clean.pickle")